In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

In [3]:
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list = []
    
    positive_path = path + filetype + "/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path + f]
        
    negative_path = path + filetype + "/neg/"
    for f in os.listdir(negative_path):
        file_list += [negative_path + f]
    
    print('read', filetype, 'files:', len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding = 'utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    
    return all_labels, all_texts   

In [4]:
y_train, train_text = read_files("train")

read train files: 25000


In [5]:
y_test,  test_text  = read_files("test")

read test files: 25000


In [6]:
token = Tokenizer(num_words = 3800)
token.fit_on_texts(train_text)

In [7]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [8]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = 380)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen = 380)

In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [10]:
model = Sequential()

In [11]:
model.add(Embedding(output_dim = 32,
                    input_dim = 3800,
                    input_length = 380))
model.add(Dropout(0.35))

In [12]:
model.add(SimpleRNN(units=16))

In [13]:
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.35))

In [14]:
model.add(Dense(units=1, activation = 'sigmoid'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_1 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 126,993
Trainable params: 126,993
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [19]:
train_history = model.fit(x_train, 
                          y_train, 
                          batch_size = 100,
                          epochs = 10,
                          verbose = 2,
                          validation_split = 0.2
                         )

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 14s - loss: 0.5398 - acc: 0.7215 - val_loss: 0.5168 - val_acc: 0.7620
Epoch 2/10
 - 13s - loss: 0.3525 - acc: 0.8560 - val_loss: 0.6012 - val_acc: 0.6132
Epoch 3/10
 - 13s - loss: 0.3063 - acc: 0.8782 - val_loss: 0.5847 - val_acc: 0.7592
Epoch 4/10
 - 13s - loss: 0.2650 - acc: 0.8977 - val_loss: 0.6954 - val_acc: 0.7048
Epoch 5/10
 - 13s - loss: 0.2336 - acc: 0.9126 - val_loss: 0.5576 - val_acc: 0.7940
Epoch 6/10
 - 13s - loss: 0.2315 - acc: 0.9088 - val_loss: 0.7854 - val_acc: 0.7310
Epoch 7/10
 - 13s - loss: 0.1920 - acc: 0.9270 - val_loss: 0.4949 - val_acc: 0.8154
Epoch 8/10
 - 13s - loss: 0.1593 - acc: 0.9421 - val_loss: 0.5175 - val_acc: 0.8336
Epoch 9/10
 - 13s - loss: 0.1279 - acc: 0.9527 - val_loss: 0.7002 - val_acc: 0.8006
Epoch 10/10
 - 13s - loss: 0.1058 - acc: 0.9612 - val_loss: 0.8587 - val_acc: 0.7836


In [20]:
scores = model.evaluate(x_test, y_test, verbose = 1)
scores[1]

25000/25000 [==============================] - 9s 345us/step


0.8398